In [1]:
import tensorflow as tf
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

 
import datetime
from scipy.io import loadmat
import scipy
from scipy import stats



Using TensorFlow backend.


In [3]:
print(tf.__version__)

1.15.0


In [4]:
#4 covariate model with rain, n20, wfps, sampidxn20 for 30, 15 and 60 day sequences 

# matx = loadmat('/Users/abhijnannath/Downloads/Xtrain.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/YTrain.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/15day_Xtrain4cov_base.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/15day_Ytrain4cov_base.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/60day_Xtrain4cov_base.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/60day_Ytrain4cov_base.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/60day_Xtrain5cov_base_soiltemp10.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/60day_Ytrain5cov_base_soiltemp10.mat')


# matx = loadmat('/Users/abhijnannath/Downloads/60day_Xtrain5cov_base_soilno3.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/60day_Ytrain5cov_base_soilno3.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/60day_Xtrain_6cov_base_soiltemp_soilno3.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/60day_Ytrain_6cov_base_soiltemp_soilno3.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/30day_Xtrain5cov_fertapp_wodayssince.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/30day_Ytrain5cov_fertapp_wodayssince.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/30day_Xtrain6cov_soiltemp10_fertapp_wodayssince.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/30day_Ytrain6cov_soiltemp10_fertapp_wodayssince.mat')
# matx = loadmat('/Users/abhijnannath/Downloads/30day_Xtrain6cov_fertapp_fertdayssinceappaug5.mat')
# maty = loadmat('/Users/abhijnannath/Downloads/30day_Ytrain6cov_fertapp_fertdayssinceappaug5.mat')

# matx = loadmat('/Users/abhijnannath/Downloads/15day_Xtrain6cov_fertapp_fertdayssinceappaug5 (1).mat')
# maty = loadmat('/Users/abhijnannath/Downloads/15day_Ytrain6cov_fertapp_fertdayssinceappaug5 (1).mat')
# matx = loadmat('/Users/abhijnannath/Downloads/30day_Xtrain7cov_soiltemp_fertapp_fertdayssinceappnewholdout (1).mat')
# maty = loadmat('/Users/abhijnannath/Downloads/30day_Ytrain7cov_soiltemp_fertapp_fertdayssinceappnewholdout (1).mat')

# matx = loadmat('/Users/abhijnannath/Downloads/60day_Xtrain7cov_soiltemp_fertapp_fertdayssinceappnewholdout (1).mat')
# maty = loadmat('/Users/abhijnannath/Downloads/60day_Ytrain7cov_soiltemp_fertapp_fertdayssinceappnewholdout (1).mat')

# matx = loadmat('/Users/abhijnannath/LSTM N2o/Outputclippingplots/Cross Valid Holdout /final trainsets for keras after matlab/sample18/Crossvalid_30day_XTrain4cov_18.mat')
# maty = loadmat('/Users/abhijnannath/LSTM N2o/Outputclippingplots/Cross Valid Holdout /final trainsets for keras after matlab/sample18/Crossvalid_30day_YTrain4cov_18.mat')

matx = loadmat('/n2o_nn/30day_XTrain7cov_soiltemp_fertapp_fertdayssinceapp_n2osf7.mat')
maty = loadmat('/n2o_nn/30day_YTrain7cov_soiltemp_fertapp_fertdayssinceapp_n2osf7.mat')


matx.keys()
maty.keys()
matx
dfx = pd.DataFrame(np.hstack((matx['XTrain1'])))
dfx.head
dfy = pd.DataFrame(np.hstack((maty['YTrain1'])))
dfy.head

arrx = dfx.values
arry = dfy.values
resultx = arrx[:,0]
resulty = arry[:,0]
 
xlist = []
ylist  = []
for i in range(len(resultx)):
    xlist.append(np.transpose(resultx[i]))
    ylist.append(np.transpose(resulty[i]))
    
Xtrain = np.asarray(xlist)
# this value is the rows/cells of the Xtrain * the SL 
# for 7 covariate, XTrain[1992,7] 1992*30=59760
# Xtrain = np.reshape(Xtrain, (49170, 4)) #for 30 day seq with 4cov
# Xtrain = np.reshape(Xtrain, (24585, 4)) #for 15 day seq with 4 cov
# Xtrain = np.reshape(Xtrain, (44880, 4)) #for 60 day seq with 4 cov
# # Xtrain = np.reshape(Xtrain, (26100, 5)) #for 60 day 5 cov base_soiltmep10 
# Xtrain = np.reshape(Xtrain, (19620, 6)) #for 60 day 5 cov base_soilno3 
# Xtrain = np.reshape(Xtrain, (74640, 5))#for 30 day 5 cov without days since but with fert applied
# Xtrain = np.reshape(Xtrain, (71460, 6))#for 30 day 6 cov without days since but with fert applied and soil temp
# Xtrain = np.reshape(Xtrain, (59910, 6))#for 30 day 6 cov but with fert applied and days since with new hold out 
# Xtrain = np.reshape(Xtrain, (51000, 6)) # 60 day as above
# Xtrain = np.reshape(Xtrain, (59640, 6)) #15 day 6 cov, day since and fert applied with new hold out 
# Xtrain = np.reshape(Xtrain, (53940, 7)) #30 day 7 cov, day since and fert applied with new hold out 
# Xtrain = np.reshape(Xtrain, (54945, 7))  #15 day as above 
# Xtrain = np.reshape(Xtrain, (53820, 7))  #15 day as above 
Xtrain = np.reshape(Xtrain, (59760, 7)) # 30 day seq repeat 
# Xtrain = np.reshape(Xtrain, (55020, 7)) # 60 day seq repeat 
#cross valid sets
#Xtrain = np.reshape(Xtrain, (89190, 4)) # cross valid set 4 --30 day  
dfx = pd.DataFrame(Xtrain)

# # Ytrain = np.asarray(ylist).reshape(49170,1)
# # Ytrain = np.asarray(ylist).reshape(24585,1)
# # Ytrain = np.asarray(ylist).reshape(44880,1) 
# # Ytrain = np.asarray(ylist).reshape(26100,1)
# # Ytrain = np.asarray(ylist).reshape(19620,1)
# # Ytrain = np.asarray(ylist).reshape(74640,1)
# # Ytrain = np.asarray(ylist).reshape(71460,1) #for 30 day 6 cov without days since but with fert applied and soil temp
# # Ytrain = np.asarray(ylist).reshape(59910,1) #for 30 day 6 cov but with fert applied and soil temp
# # Ytrain = np.asarray(ylist).reshape(51000,1) #for 60 day as above 
# # Ytrain = np.asarray(ylist).reshape(59640,1) #15 day 6 cov, day since and fert applied with new hold out 
# # Ytrain = np.asarray(ylist).reshape(53940,1) #30 day 7 cov, day since and fert applied + soil temp 10  with new hold out 
# # Ytrain = np.asarray(ylist).reshape(54945,1) #15 day as above covariates
# # Ytrain = np.asarray(ylist).reshape(53820,1) #60 day as above covariates

Ytrain = np.asarray(ylist).reshape(59760,1) #30 day repeat 
# # Ytrain = np.asarray(ylist).reshape(55020,1) #60 day repeat 

# Ytrain = np.asarray(ylist).reshape(89190,1) # cross valid set 4 --30 day  
dfy = pd.DataFrame(Ytrain)

# dfx.columns = ['sampn2o', 'rainseq','soilseq','NO3','wfpsseq','n2o']
dfx.columns = ['sampn2o', 'rainseq','fertseq','wfpsseq','dayssinceappseq','soiltemp10','n2o']
# dfx.columns = ['sampn2o', 'rainseq','wfpsseq','n2o']
df_targets = pd.DataFrame(dfy)
df_targets.columns = ['target n2o']

x_data = dfx.values
y_data = df_targets.values
num_data = len(x_data)
num_data 

#new training and testing for 30 day sequences and with lesser hidden neurons. Comment it out to go back to the original 60 day sequence batches

scaler = StandardScaler()
print(scaler.fit(y_data))
print(scaler.mean_)
y_data =scaler.transform(y_data)
y_data.shape
     
 
# x_data = x_data.reshape((1639,15,4)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((1639,15,1))

# x_data = x_data.reshape((748,60,4)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((748,60,1))


# x_data = x_data.reshape((435,60,5)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((435,60,1)) #for 60 day seq for soiltemp10 +base


# x_data = x_data.reshape((327,60,6)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((327,60,1)) #for 60 day soil no 3

# x_data = x_data.reshape((2488,30,5)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((2488,30,1)) #for 30 day 5 cov without days since but with fert applied

# x_data = x_data.reshape((2382,30,6)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((2382,30,1)) #for 30 day 6 cov without days since but with fert applied + soil temp 10 

# x_data = x_data.reshape((1997,30,6)) #change it to n for n-day sequences/ time steps
# y_data = y_data.reshape((1997,30,1)) #for 30 day 6 cov with fert apllied and days since app 


# x_data = x_data.reshape((850,60,6)) #change it to 15 for n-day sequences/ time steps with fert apllied and days since app 
# y_data = y_data.reshape((850,60,1))

# x_data = x_data.reshape((3976,15,6))  #15 day 6 cov, day since and fert applied with new hold out 
# y_data = y_data.reshape((3976,15,1))


# x_data = x_data.reshape((1798,30,7))  #30 day 7 cov, day since and fert applied + soil temp 10 with new hold out 
# y_data = y_data.reshape((1798,30,1))

# x_data = x_data.reshape((3663,15,7))  #15 day 7 cov, day since and fert applied + soil temp 10 with new hold out 
# y_data = y_data.reshape((3663,15,1))

# x_data = x_data.reshape((897,60,7))  #60 day 7 cov, day since and fert applied + soil temp 10 with new hold out 
# y_data = y_data.reshape((897,60,1))

x_data = x_data.reshape((1992,30,7))  #30 day 7 cov repeat after mistake , day since and fert applied + soil temp 10 with new hold out 
y_data = y_data.reshape((1992,30,1))

# x_data = x_data.reshape((917,60,7))  #30 day 7 cov repeat after mistake , day since and fert applied + soil temp 10 with new hold out 
# y_data = y_data.reshape((917,60,1))


# x_data = x_data.reshape((2973,30,4))  # cross valid set 4 
# y_data = y_data.reshape((2973,30,1))

x_data.shape, y_data.shape
 
#try standarsizing the target variable as the model weights were exploding due to the large variation in the y  or the target data
 





 

 



 
    




StandardScaler()
[9.98878301]


((1992, 30, 7), (1992, 30, 1))

In [5]:
len(xlist)

1992

In [6]:
#LSTM architecture here    # changing the architecture to reduce hidden layer neurons to prevent overfitting with 25 neurons, 50 precent dropout
#and 30 time steps as input sequences. 
from keras.layers import Dropout

# from tensorflow.keras import layers # new for this version 

model = Sequential()
# model = keras.Sequential() # tf2.3

# is this needed...? 
# # Add an Embedding layer expecting input vocab of size 1000, and
# # output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64))

# for tensorflow 2.3
# https://www.tensorflow.org/guide/keras/rnn
# model.add(layers.LSTM(128,activation='tanh',
#                return_sequences=True,
#                ) )
# need to figure out the input_shape parameter 

# print(model)

# need to see about tensorflow syntax, AN used 1.15
model.add(LSTM(50,activation='tanh',
              return_sequences=True,  
             input_shape=(30,7)))

# model.add(LSTM(32, activation='tanh', return_sequences=True))

  # if doing dropout 0.4 then need both 50 and 32 neuron 
    # otherwise dropout 0.2, 0.3 use just 50 layer 
    
model.add(Dropout(0.2))
##model.add(Dense(num_y_signals, activation='linear'))
model.add(Dense(1))
# # model.add(Dense(1, activation='relu'))
 
 
 
# if False:
#     from tensorflow.python.keras.initializers import RandomUniform

#     # Maybe use lower init-ranges.
#     init = RandomUniform(minval=-0.05, maxval=0.05)

#     model.add(Dense(num_y_signals,
#                     activation='linear',
#                     kernel_initializer=init))

 


 
 





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# optimizer = adam(lr=1e-2)
# optimizer
# from keras.optimizers import adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
# opt = adam(lr=0.001)

opt = RMSprop(lr=0.001)


In [8]:
model.compile(loss='mse', optimizer=opt)

In [9]:
# model.build()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 50)            11600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 50)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 30, 1)             51        
Total params: 11,651
Trainable params: 11,651
Non-trainable params: 0
_________________________________________________________________


In [10]:
from keras.callbacks import ModelCheckpoint
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [11]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=20, verbose=1) #initially patience was 5 for the 60 sequence model, now let's make iit 20 to give the model more attempts at convergence 

In [12]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [13]:
 callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=20,
                                       verbose=1)

In [14]:
# callbacks = [callback_early_stopping,
#              callback_checkpoint,
#              callback_tensorboard,
#              callback_reduce_lr]

x_data.shape
y_data.shape

 
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', restore_best_weights=True)
# mcp_save = ModelCheckpoint(filepath = '/Users/abhijnannath/crossvalid_18_30day_mdlwts_4covariate_base_Do20_lr001_epoch500_rmsprop_new_monitorloss_aug5.hdf5', save_best_only=True, monitor='loss', mode='min')
mcp_save = ModelCheckpoint(filepath = '/n2o_nn/LSTM_June2022/4_Python_LSTMmodels_hdf5/30day_7covariate_Do20_lr001_epoch500_rmsprop_new_monitorloss_n2osf7.hdf5', save_best_only=True, monitor='loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-4, mode='min')
callbacks = [mcp_save]


In [15]:
# from tensorflow.python.keras.utils.data_utils import Sequence 
# model.fit(x_batch,y_batch, epochs=1, steps_per_epoch=5, validation_data=validation_data)

# model.fit(x_batch,y_batch,
#           epochs=200,batch_size=16, validation_split=0.1, verbose=1)  #this is for the 60 day sequence. 

#now let's do this for 30 day sequence with callbacks 
history_fert_dayssince_4cov_lr001_rmsprop_lstm_dro20 = model.fit(x_data,y_data,
          epochs=500,batch_size=16, validation_split=0.1, verbose=1, callbacks = callbacks)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1792 samples, validate on 200 samples
Epoch 1/500





1792/1792 [==============================] - 2s 1ms/step - loss: 0.7752 - val_loss: 0.4406
Epoch 2/500
1792/1792 [==============================] - 1s 833us/step - loss: 0.5819 - val_loss: 0.5778
Epoch 3/500
1792/1792 [==============================] - 2s 850us/step - loss: 0.5550 - val_loss: 0.4143
Epoch 4/500
1792/1792 [==============================] - 1s 835us/step - loss: 0.5031 - val_loss: 0.6559
Epoch 5/500
1792/1792 [==============================] - 2s 1ms/step - loss: 0.5036 - val_loss: 0.4049
Epoch 6/500
1792/1792 [==============================] - 2s 1ms/step - loss: 0.4829 - val_loss: 0.5274
Epoch 7/500
1792/1792 [==============================] - 2s 1ms/step - loss: 0.4736 - val_loss: 0.4406
Epoch 8/500
1792/1792 [==============================] - 2s 1ms/step - loss: 0.4642 - val_loss: 0.5857
Epoch 9/500
1792/1792